## Data Cleaning

#### Import modules

In [1]:
import os
import pandas as pd

#### Define paths and load data

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
PATH = 'raw_data'

In [4]:
OUTPUT_PATH = 'outputs'

### Election results

#### Load data

In [5]:
results = pd.read_csv(os.path.join(PATH, 'HarvardData_countypres_2000-2020.csv'))
results = results[results['party'].isin(['DEMOCRAT', 'REPUBLICAN'])]
grpby_cols = ['year', 'state', 'state_po', 'county_name', 'county_fips', 'office',
              'candidate', 'party', 'totalvotes', 'version']
results = results.groupby(grpby_cols, as_index=False).sum()

In [6]:
results['pctvotes'] = results['candidatevotes'] / results['totalvotes']
results['office'] = 'PRESIDENT'  # for consistency

In [7]:
results.state.unique()

array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA',
       'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA',
       'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA',
       'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND',
       'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI',
       'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
       'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA',
       'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA',
       'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE',
       'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'], dtype=object)

#### Pivot table

In [8]:
index_cols = ['year', 'state', 'state_po', 'county_name', 
              'county_fips', 'office']

In [9]:
pivoted_results = pd.pivot_table(results, index=index_cols, values=['pctvotes', 'candidatevotes'], columns=['party']).reset_index()
pivoted_results.columns.name = ''
pivoted_results.columns = [(x[0] + '_' + x[1]).lower() if x[1] != '' else x[0] for x in  pivoted_results.columns.values]
a = pivoted_results['candidatevotes_democrat'] > pivoted_results['candidatevotes_republican']
pivoted_results['winner'] = ['democrat' if x else 'republican' for x in a]
pivoted_results = pivoted_results[['year', 'state', 'county_name', 'county_fips', 'candidatevotes_democrat',
                                   'candidatevotes_republican', 'pctvotes_democrat', 'pctvotes_republican', 'winner']]

In [10]:
pivoted_results.head()

,year,state,county_name,county_fips,candidatevotes_democrat,candidatevotes_republican,pctvotes_democrat,pctvotes_republican,winner
0,2000,ALABAMA,AUTAUGA,1001.0,4942.0,11993.0,0.287192,0.696943,republican
1,2000,ALABAMA,BALDWIN,1003.0,13997.0,40872.0,0.247822,0.723654,republican
2,2000,ALABAMA,BARBOUR,1005.0,5188.0,5096.0,0.499086,0.490236,democrat
3,2000,ALABAMA,BIBB,1007.0,2710.0,4273.0,0.381636,0.601746,republican
4,2000,ALABAMA,BLOUNT,1009.0,4977.0,12667.0,0.276915,0.704779,republican


Export election results

In [11]:
for year in pivoted_results['year'].unique():
    df_temp = pivoted_results[pivoted_results['year'] == year]
    df_temp.to_csv(os.path.join(OUTPUT_PATH, f'results_{year}.csv'), index=False)

### Unemployment data

In [12]:
unemployment = pd.read_excel(os.path.join(PATH, 'ERS_Unemployment.xlsx'), header=4)

In [13]:
rates_cols = ['Unemployment_rate_2000',
              'Unemployment_rate_2004',
              'Unemployment_rate_2008',
              'Unemployment_rate_2012',
              'Unemployment_rate_2016',
              'Unemployment_rate_2020']

In [14]:
area_cols = ['State', 'Area_name', 'FIPS_Code']

In [15]:
unemployment_ey = unemployment[area_cols + rates_cols]

In [16]:
unemployment_ey.sample(2)

,State,Area_name,FIPS_Code,Unemployment_rate_2000,Unemployment_rate_2004,Unemployment_rate_2008,Unemployment_rate_2012,Unemployment_rate_2016,Unemployment_rate_2020
2480,TN,"Campbell County, TN",47013,6.0,5.9,7.7,11.2,6.9,7.6
2037,ND,"Cavalier County, ND",38019,3.4,3.6,3.0,3.0,2.7,3.5


In [17]:
unemployment_ey_melted = pd.melt(unemployment_ey, 
                                 id_vars=area_cols, 
                                 value_vars=rates_cols, 
                                 value_name='unemployment',
                                 var_name='year')
unemployment_ey_melted['year'] = [x.split('_')[2] for x in unemployment_ey_melted['year']]
unemployment_ey_melted['year'] = unemployment_ey_melted['year'].astype(int)
unemployment_ey_melted.rename(columns={'FIPS_Code': 'county_fips'}, inplace=True)
unemployment_ey_melted = unemployment_ey_melted[['county_fips', 'year', 'unemployment']]

In [18]:
unemployment_ey_melted.sample()

,county_fips,year,unemployment
4345,21113,2004,4.4


Export unemployment data

In [19]:
for year in unemployment_ey_melted['year'].unique():
    
    df_temp = unemployment_ey_melted[unemployment_ey_melted['year'] == year]
    df_temp.to_csv(os.path.join(OUTPUT_PATH, f'unemployment_{year}.csv'), index=False)

#### Population

In [20]:
population = pd.read_excel(os.path.join(PATH, 'ERS_PopulationEstimates.xls'), header=2)

In [21]:
population.sample()

,FIPStxt,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,POP_ESTIMATE_2010,POP_ESTIMATE_2011,POP_ESTIMATE_2012,POP_ESTIMATE_2013,POP_ESTIMATE_2014,POP_ESTIMATE_2015,POP_ESTIMATE_2016,POP_ESTIMATE_2017,POP_ESTIMATE_2018,POP_ESTIMATE_2019,N_POP_CHG_2010,N_POP_CHG_2011,N_POP_CHG_2012,N_POP_CHG_2013,N_POP_CHG_2014,N_POP_CHG_2015,N_POP_CHG_2016,N_POP_CHG_2017,N_POP_CHG_2018,N_POP_CHG_2019,Births_2010,Births_2011,Births_2012,Births_2013,Births_2014,Births_2015,Births_2016,Births_2017,Births_2018,Births_2019,Deaths_2010,Deaths_2011,Deaths_2012,Deaths_2013,Deaths_2014,Deaths_2015,Deaths_2016,Deaths_2017,Deaths_2018,Deaths_2019,NATURAL_INC_2010,NATURAL_INC_2011,NATURAL_INC_2012,NATURAL_INC_2013,NATURAL_INC_2014,NATURAL_INC_2015,NATURAL_INC_2016,NATURAL_INC_2017,NATURAL_INC_2018,NATURAL_INC_2019,INTERNATIONAL_MIG_2010,INTERNATIONAL_MIG_2011,INTERNATIONAL_MIG_2012,INTERNATIONAL_MIG_2013,INTERNATIONAL_MIG_2014,INTERNATIONAL_MIG_2015,INTERNATIONAL_MIG_2016,INTERNATIONAL_MIG_2017,INTERNATIONAL_MIG_2018,INTERNATIONAL_MIG_2019,DOMESTIC_MIG_2010,DOMESTIC_MIG_2011,DOMESTIC_MIG_2012,DOMESTIC_MIG_2013,DOMESTIC_MIG_2014,DOMESTIC_MIG_2015,DOMESTIC_MIG_2016,DOMESTIC_MIG_2017,DOMESTIC_MIG_2018,DOMESTIC_MIG_2019,NET_MIG_2010,NET_MIG_2011,NET_MIG_2012,NET_MIG_2013,NET_MIG_2014,NET_MIG_2015,NET_MIG_2016,NET_MIG_2017,NET_MIG_2018,NET_MIG_2019,RESIDUAL_2010,RESIDUAL_2011,RESIDUAL_2012,RESIDUAL_2013,RESIDUAL_2014,RESIDUAL_2015,RESIDUAL_2016,RESIDUAL_2017,RESIDUAL_2018,RESIDUAL_2019,GQ_ESTIMATES_BASE_2010,GQ_ESTIMATES_2010,GQ_ESTIMATES_2011,GQ_ESTIMATES_2012,GQ_ESTIMATES_2013,GQ_ESTIMATES_2014,GQ_ESTIMATES_2015,GQ_ESTIMATES_2016,GQ_ESTIMATES_2017,GQ_ESTIMATES_2018,GQ_ESTIMATES_2019,R_birth_2011,R_birth_2012,R_birth_2013,R_birth_2014,R_birth_2015,R_birth_2016,R_birth_2017,R_birth_2018,R_birth_2019,R_death_2011,R_death_2012,R_death_2013,R_death_2014,R_death_2015,R_death_2016,R_death_2017,R_death_2018,R_death_2019,R_NATURAL_INC_2011,R_NATURAL_INC_2012,R_NATURAL_INC_2013,R_NATURAL_INC_2014,R_NATURAL_INC_2015,R_NATURAL_INC_2016,R_NATURAL_INC_2017,R_NATURAL_INC_2018,R_NATURAL_INC_2019,R_INTERNATIONAL_MIG_2011,R_INTERNATIONAL_MIG_2012,R_INTERNATIONAL_MIG_2013,R_INTERNATIONAL_MIG_2014,R_INTERNATIONAL_MIG_2015,R_INTERNATIONAL_MIG_2016,R_INTERNATIONAL_MIG_2017,R_INTERNATIONAL_MIG_2018,R_INTERNATIONAL_MIG_2019,R_DOMESTIC_MIG_2011,R_DOMESTIC_MIG_2012,R_DOMESTIC_MIG_2013,R_DOMESTIC_MIG_2014,R_DOMESTIC_MIG_2015,R_DOMESTIC_MIG_2016,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_DOMESTIC_MIG_2019,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018,R_NET_MIG_2019
2693,48251,TX,Johnson County,1.0,1.0,1.0,1.0,0.0,150934,150956,151264,152094,153415,154535,156733,159345,162763,167012,171121,175817,308.0,830.0,1321.0,1120.0,2198.0,2612.0,3418.0,4249.0,4109.0,4696.0,484.0,1919.0,1964.0,1938.0,1953.0,2107.0,2092.0,2093.0,2153.0,2155.0,228.0,1284.0,1172.0,1269.0,1311.0,1332.0,1369.0,1395.0,1462.0,1464.0,256.0,635.0,792.0,669.0,642.0,775.0,723.0,698.0,691.0,691.0,22.0,59.0,20.0,2.0,-14.0,27.0,27.0,8.0,-11.0,-18.0,43.0,140.0,542.0,474.0,1569.0,1804.0,2664.0,3536.0,3429.0,4018.0,65.0,199.0,562.0,476.0,1555.0,1831.0,2691.0,3544.0,3418.0,4000.0,-13.0,-4.0,-33.0,-25.0,1.0,6.0,4.0,7.0,0.0,5.0,2644.0,2645.0,2643.0,2648.0,2678.0,2696.0,2709.0,2698.0,2676.0,2601.0,2596.0,12.651718,12.857232,12.586459,12.548672,13.332152,12.989432,12.693503,12.734634,12.422969,8.465246,7.672442,8.241598,8.423609,8.428299,8.500255,8.460314,8.647485,8.439548,4.186473,5.18479,4.344861,4.125063,4.903853,4.489178,4.233189,4.087149,3.983421,0.388979,0.130929,0.012989,-0.089955,0.170844,0.167646,0.048518,-0.065063,-0.103765,0.923002,3.548177,3.078422,10.081345,11.414904,16.541036,21.444925,20.28196,23.16264,1.311981,3.679106,3.091411,9.99139,11.585748,16.708682,21.493442,20.216897,23.058875


We obtained the rates for natural increase in population, international migration, domestic migration, and net migration for the years 2012, 2016, and we substituted the value of 2019 for 2020. 

In [22]:
years_population = ['2012', '2016', '2019']

In [23]:
rate_nat_inc_cols = ['R_NATURAL_INC_' + str(x) for x in years_population]
rate_int_mig_cols = ['R_INTERNATIONAL_MIG_' + str(x) for x in years_population]
rate_dom_mig_cols = ['R_DOMESTIC_MIG_' + str(x) for x in years_population]
rate_net_mig_cols = ['R_NET_MIG_' + str(x) for x in years_population]

In [24]:
population_dict = {'rate_natural_increase_population': rate_nat_inc_cols,
                   'rate_international_migration': rate_int_mig_cols,
                   'rate_domestic_migration': rate_dom_mig_cols,
                   'rate_net_migration': rate_net_mig_cols}

In [25]:
for key in population_dict.keys():
    
    cols = population_dict[key]
    temp2 = pd.melt(population[['FIPStxt'] + cols], 
                    id_vars=['FIPStxt'], 
                    value_vars=cols, 
                    value_name=key,
                    var_name='year')
    temp2['year'] = [x.split("_")[3] for x in temp2['year']]
    temp2['year'] = ['2020' if x == '2019' else x for x in temp2['year']]
    
    for year in temp2['year'].unique():
        df_temp = temp2[temp2['year'] == year]
        df_temp.rename(columns={'FIPStxt': 'county_fips'}, inplace=True)
        df_temp.drop(['year'], inplace=True, axis=1)
        df_temp.to_csv(os.path.join(OUTPUT_PATH, f'{key}_{year}.csv'), index=False)
        

C:\Users\isnek\envs\algo1\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\isnek\envs\algo1\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [26]:
natural_inc_year = pd.melt(population[['FIPStxt'] + rate_nat_inc_cols], 
                           id_vars=['FIPStxt'], 
                           value_vars=rate_nat_inc_cols, 
                           value_name='natural_increase_in_pop',
                           var_name='year')

natural_inc_year['year'] = [x.split("_")[3] for x in natural_inc_year['year']]
natural_inc_year['year'] = ['2020' if x == '2019' else x for x in natural_inc_year['year']]

#### Facebook

In [27]:
facebook = pd.read_csv(os.path.join(PATH, 'Facebook_Cornell_ERS_SocialConnectednessIndex.csv'))
facebook.rename(columns={'FIPS': 'county_fips'}, inplace=True)
facebook = facebook[['county_fips', 'sh050m', 'sh100m', 'sh500m']]

In [28]:
facebook.to_csv(os.path.join(OUTPUT_PATH, f'facebook_2014.csv'), index=False)

#### Social Capital

In [29]:
social_capital = pd.read_csv(os.path.join(PATH, 'PennState_AgriScienceDept_Religion_and_Social_Capital.csv'))
social_capital = social_capital.iloc[:, :18]
social_capital.drop(['County_Name', 'pop2014'], inplace=True, axis=1)
social_capital.rename(columns={'FIPS': 'county_fips'}, inplace=True)

In [30]:
social_capital.to_csv(os.path.join(OUTPUT_PATH, f'social_capital.csv'), index=False)

#### Infra

In [31]:
infra = pd.read_csv(os.path.join(PATH, 'HarvardData_streets_intersections.csv'))
infra.rename(columns={'geoid': 'county_fips'}, inplace=True)
infra = infra[['county_fips', 'streets_per_node_avg']]

In [32]:
infra.head()

,county_fips,streets_per_node_avg
0,1001,2.455156
1,1003,2.501281
2,1005,2.526385
3,1007,2.398802
4,1009,2.334520


In [33]:
infra.to_csv(os.path.join(OUTPUT_PATH, f'infra.csv'), index=False)

#### Broadband

In [34]:
broadband = pd.read_csv(os.path.join(PATH, 'FCC_Health_and_Broadband.csv'))
broadband.rename(columns={'geography_id': 'county_fips'}, inplace=True)
broadband = broadband[['county_fips', 'pctpopwbbacc']]

In [35]:
broadband.head()

,county_fips,pctpopwbbacc
0,21109,99.6
1,37005,87.7
2,13123,95.1
3,20015,73.3
4,6025,85.7


In [36]:
broadband.to_csv(os.path.join(OUTPUT_PATH, f'broadband.csv'), index=False)